需要修改的部分

In [17]:
#工作文件路径
path = '/Users/cecilia1/Desktop/UIUC暑研/wwpie/'

#

In [1]:
# sentence tree class
from collections import deque


class SnTree(object):
    # global cc_words
    # cc_words=['and','or','but']
    def __init__(self, sentence_tree):
        # self.order = []     #visited order
        self.tree = sentence_tree
        self.root = -1
        self.subtrees = {}
        self.treeEncode = {}

    def set_root(self, r_id):
        self.root = r_id

    def encode_tree(self, r, coding):
        self.treeEncode[r] = coding
        # print(r, self.tree[r][-1])
        for i, p in enumerate(self.tree[r][-1]):
            # print(p)
            new_coding = coding + chr(ord('A') + i)
            self.encode_tree(r=p, coding=new_coding)

    def split_conj(self):
        search_queue = deque()
        search_queue += list(self.subtrees.items())
        # print('seachque',search_queue)
        # split on each subtree
        while search_queue:
            tempSen = list(search_queue.popleft()[1])
            sentence = sorted(tempSen)
            # print('sentence',sentence)
            not_position = -1
            neither_position = -1
            for i, word in enumerate(sentence):
                if type(word) == float:
                    break
                # if two vb conjunction
                # second vb's dep is conj and its parent is verb
                if self.tree[word][5] == 'not':
                    not_position = word
                if self.tree[word][5] == 'neither':
                    neither_position = word

                if self.tree[word][2] == 'VERB' and self.tree[word][3] == 'conj' and self.tree[self.tree[word][4]][
                    2] == 'VERB' and self.tree[word][4] in sentence:
                    verb1 = self.tree[word][4]
                    verb2 = word
                    break_point = -1

                    difSub = False
                    difObj = False
                    noObj1 = True
                    noObj2 = True
                    for w2 in self.tree[word][-1]:
                        if 'nsubj' in self.tree[w2][3]:  # do not share the same subject
                            difSub = True
                            # print('found subject for the second verb', w2)
                        if self.tree[w2][3] == 'dobj':
                            noObj2 = False  # has object

                    for w1 in self.tree[self.tree[word][4]][-1]:

                        if self.tree[w1][3] == 'dobj':
                            noObj1 = False  # has object
                            if w1 < word:  # do not share the same object
                                difObj = True
                                # print('found object for the first verb',w1)
                        if w1 > verb1 and w1 < word and (self.tree[w1][2] == 'ADV' or self.tree[w1][2] == 'ADP'):
                            difObj = True
                        if self.tree[w1][3] == 'cc' and w1 < word and w1 in sentence and w1 > verb1:
                            break_point = w1
                            # print('break point', break_point)

                    while (False):
                        # while(i>0 and break_point<0):
                        i -= 1
                        if self.tree[sentence[i]][1] == ',' or self.tree[sentence[i]][1] == ';':
                            break_point = sentence[i]
                            # print('break point', break_point)
                    if break_point < 0:
                        break
                    sent1 = []
                    sent2 = []
                    # if difSub and (difObj or (noObj1 and noObj2)):
                    if difSub:
                        # not share subject or object. safe to split
                        # split at cc
                        for w in sentence:
                            if w < break_point:
                                sent1.append(w)
                            elif w == break_point:
                                continue
                            else:
                                sent2.append(w)
                    # elif difSub==False and (difObj or (noObj1 and noObj2)):
                    else:
                        # same subject, different object
                        for j, w in enumerate(sentence):
                            if w < verb1:
                                if (w == not_position and self.tree[sentence[j + 1]][1] == 'only') or (
                                        sentence[j - 1] == not_position and self.tree[w][1] == 'only'):
                                    # not only, but also
                                    continue
                                if self.tree[w][1] == 'both' or self.tree[w][1] == 'either':
                                    # not only, but also
                                    continue
                                elif w == not_position and self.tree[break_point][1] == 'but' or (
                                            self.tree[w][3] == 'advmod' and self.tree[w][4] == verb1 and sentence.index(
                                        verb2) - sentence.index(break_point) > 1):
                                    # not , but
                                    # adverb
                                    sent1.append(w)
                                elif w == not_position and self.tree[break_point][1] == 'or':
                                    # not , or
                                    sent1.append(w)
                                    sent2.append(w)
                                elif w == neither_position:
                                    # neither_nor/or
                                    sent1.append(w + 0.5)
                                    sent2.append(w + 0.5)
                                else:
                                    sent1.append(w)
                                    sent2.append(w)
                            elif w < break_point:
                                sent1.append(w)
                            elif w == break_point:
                                continue
                            else:
                                if (self.tree[w][1] == 'also' and self.tree[break_point][1] == 'but' and sentence[
                                        j - 1] == break_point):
                                    continue
                                sent2.append(w)

                    self.subtrees[verb1] = set(sent1)
                    self.subtrees[verb2] = set(sent2)
                    # elif difSub :
                    # print(verb2, sent2)
                    search_queue += [(verb2, set(sent2))]
                    break

                    # second vb's dep is conj and its parent's dep is acomp
                    # if word[1][3]=='conj' and self.tree[word[1][4]][3]=='acomp':

    def split_noun(self, r):
        # print('start')
        if r != -1:
            search_queue = deque()
            search_queue.append(r)
            self.subtrees[r] = set()
            # print(search_queue)
        else:
            print('root is None')
            return -1

        while search_queue:
            person = search_queue.popleft()
            # print(person)

            if (not person in self.subtrees[r]) and (person in self.tree.keys()):
                if (self.tree[person][2] != 'NOUN' and self.tree[person][2] != 'PROPN') or person == r:
                    search_queue += self.tree[person][-1]
                    self.subtrees[r].add(person)
                else:
                    self.subtrees[r].add(person)
                    self.split_noun(person)

    def concentrate_subtrees(self, windowsize_min, windowsize_max):
        keys = list(self.subtrees.keys())
        for k in reversed(keys):
            if len(self.subtrees[k]) < windowsize_min:
                # print(k,self.subtrees[k])
                for key in self.subtrees:
                    if k in self.subtrees[key]:
                        # print(key, self.subtrees[key])
                        mergeLen = len(self.subtrees[key]) + len(self.subtrees[k])
                        if mergeLen < windowsize_max or len(self.subtrees[k]) < 3:
                            self.subtrees[key].update(self.subtrees[k])
                            if k != key:
                                del self.subtrees[k]
                        break

    def subsentecne(self):
        subs = []
        # print(self.subtrees)
        for key in self.subtrees.keys():
            # for key in sorted(self.subtrees.keys()):
            subs.append([])
            sentence_by_id = sorted(list(self.subtrees[key]))
            subs[-1].append(self.tree[key][5])
            subs[-1].append('\t')
            for w in sentence_by_id:
                if type(w) == float:
                    subs[-1].append('not')
                else:
                    if 'CHEMICAL' in self.tree[w][1] or 'GENE' in self.tree[w][1] or 'SPECIES' in self.tree[w][
                        1] or 'DISEASE' in self.tree[w][1]:
                        subs[-1].append(self.tree[w][1])
                    else:
                        subs[-1].append(self.tree[w][5])
                        # print(self.tree[w][1])
            subs[-1] = '$'.join(subs[-1])
        return subs

    def subencode(self):
        # print(self.treeEncode)
        subs = []
        # for key in sorted(self.subtrees.keys()):
        for key in self.subtrees.keys():
            subs.append([])
            sentence_by_id = sorted(list(self.subtrees[key]))
            subs[-1].append(self.treeEncode[key])
            # print(self.treeEncode[key])
            subs[-1].append('\t')
            for w in sentence_by_id:
                if type(w) == float:
                    subs[-1].append(self.treeEncode[key] + 'A')
                else:
                    subs[-1].append(self.treeEncode[w])
                    # print(self.treeEncode[w])

            subs[-1] = '$'.join(subs[-1])
        return subs

    def subpos(self):
        subs = []
        for key in self.subtrees.keys():
            # for key in sorted(self.subtrees.keys()):
            subs.append([])
            sentence_by_id = sorted(list(self.subtrees[key]))
            subs[-1].append(self.tree[key][2])
            subs[-1].append('\t')
            for w in sentence_by_id:
                if type(w) == float:
                    subs[-1].append('not')
                else:
                    subs[-1].append(self.tree[w][2])

            subs[-1] = ' '.join(subs[-1])
        return subs

In [2]:
merged_data_dict=dict()
merged_data=[]
read_file =['ner.']
ner_types=['DISEASE','GENE', 'CHEMICAL','SPECIES']
entity_dict=dict()
entity_id=0
j=0
line_number=0
line_id=[]
for f in read_file:
    thefile = open(path + 'Corpus_subset.txt','r')
    for line in thefile:
        line_number=line_number+1
        sn = line.rstrip('.\n')           
        arr=sn.split(' ')
        tokens=[]
        if len(arr)<5:
            continue
        for i in range(1,len(arr)):
            for ner in ner_types:
                start_with_dash=False
                if ner in arr[i]:
                    punct=''
                    if arr[i][-2:]=='),' or arr[i][-2:]==');':
                        punct=arr[i][-2:]
                    elif arr[i][-1]==',' or arr[i][-1]==')' or arr[i][-1]=='"' or arr[i][-1]==';':
                        punct=arr[i][-1]

                    centity=arr[i].split('-')
                    if not centity[0]:
                        start_with_dash=True
                        centity.pop(0)
                    temp_entity=centity[0].split('_')[0] + str(j)

                    if ner not in temp_entity:
                        for nerner in ner_types:
                            if nerner in temp_entity:
                                temp_entity=temp_entity.replace(nerner,ner)

                    t1=ner+ str(j)


                    if len(centity)>1:
                        original_entity='-'.join(centity[0:-1])
                        arr[i]=temp_entity + '_'+centity[-1]
                    else:
                        arr[i]=temp_entity + punct
                        if start_with_dash:
                            arr[i]='-'+arr[i]
                        original_entity=centity
                    entity_dict[t1]=original_entity


                    j+=1
                    break
            if '-' in arr[i]:
                t = arr[i].replace('-','_')
                tokens.append(t)
            else:
                tokens.append(arr[i])
        merged_data.append(' '.join(tokens))
        line_id.append(str(line_number))
print('reading_in:done')

reading_in:done


In [3]:
import pickle
import spacy
nlp = spacy.load('en_core_web_sm')
write=True
f = 'subSentence.'
subOutput = open(path + f + 'txt', 'w')
h = 'subPOS.'
posOutput = open(path + h + 'txt', 'w')
l = 'subEncode.'
encodeOutput = open(path + l + 'txt', 'w')

subSentence=[]
subSentence_pos=[]
subSentence_encode=[]
for j in range(len(merged_data)):
    doc = nlp(merged_data[j])
    sentence_tree={}
    sentence_visit=[]
    for i, token in enumerate(doc):
        sentence_tree[token.idx]=[i, token.text,token.pos_, token.dep_,token.head.idx,token.lemma_, [child.idx for child in token.ancestors],[child.idx for child in token.children]]
        if token.dep_ =='ROOT':
            root_id=token.idx

    sentence_try=SnTree(sentence_tree)

    sentence_try.root=root_id
    sentence_try.encode_tree(root_id,'A')
    sentence_try.split_noun(root_id)
    sentence_try.split_conj()
    subSentence += sentence_try.subsentecne()
    if j % 2000 == 0:
        print(j)
    subSentence_pos += sentence_try.subpos()
    subSentence_encode += sentence_try.subencode()
    if write:
        subOutput.write(line_id[j] + '\n')
        for subs in sentence_try.subsentecne():
            subOutput.write(subs + '\n')        
        subOutput.write('\n')
        posOutput.write(line_id[j] + '\n')
        for subs in sentence_try.subpos():
            posOutput.write(subs + '\n')        
        posOutput.write('\n')
        encodeOutput.write(line_id[j] + '\n')
        for subs in sentence_try.subencode():
            encodeOutput.write(subs + '\n')        
        encodeOutput.write('\n')
        
subOutput.close()
posOutput.close()
encodeOutput.close()



0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
84000
86000
88000
90000
92000
94000
96000
98000
100000
102000
104000
106000
108000
110000
112000
114000


In [4]:
def frequentPatternMining(tokens, tokensEncode, threshold, maxlen):
    dict_token = {}
    all_token={}
    
    tokensNumber = len(tokens)
    for i in range(tokensNumber):
        token = tokens[i]
        if token == '$':
            continue
        if token in dict_token:
            dict_token[token][0].append(i)
            dict_token[token][1].append(1)
        else:
            dict_token[token] = [[i],[1]]
    print ("# of distinct tokens = ", len(dict_token))

    token_pattern={}

    frequentPatterns = []
    patternLength = 1
    while (len(dict_token) > 0):
        if patternLength > maxlen:
            break
        print ("working on length = ", patternLength)
        patternLength += 1
        newDict = {}
        for pattern, positions_tf in dict_token.items():
            positions=positions_tf[0]
            valid=positions_tf[1]
            occurrence = len(positions)
            if occurrence >= threshold:
                
                if patternLength>2 and sum(valid)>threshold:
                    for j, p in enumerate(positions):
                        if valid[j]>0:
                            if p in token_pattern:
                                token_pattern[p].append(pattern)
                            else:
                                token_pattern[p]=[pattern]
                                
                    
                    frequentPatterns.append([pattern,occurrence])
                    all_token[pattern]=positions
                
                for i in positions:
                    if i + 1 < tokensNumber:
                        if tokens[i + 1] == '$':
                            continue
                        newPattern = pattern + " " + tokens[i + 1]
                        invalid=False
                        pattern_tree=tokensEncode[i-patternLength+2: i+2]
                        pattern_root=min(pattern_tree, key=len)
                        for t in pattern_tree:
                            if t[0:-1] in pattern_tree or t==pattern_root:
                                continue
                            else:
                                invalid=True
                                
                        if invalid:
                            new_valid=0
                        else:
                            new_valid=1
                         
                        if newPattern in newDict:
                            newDict[newPattern][0].append(i + 1)
                            newDict[newPattern][1].append(new_valid) 
                        else:
                            newDict[newPattern] = [[i + 1], [new_valid]]


                    
                    
        dict_token.clear()
        dict_token = newDict

    return sorted(frequentPatterns, key=lambda x:(x[0],x[1])), all_token, token_pattern

In [34]:
f = 'subSentence.'
subOutput = open(path + f + 'txt', 'r')
tokens=[]
main_words=[]
for line in subOutput:
    sn = line.rstrip('\n').split('$\t$')
    if len(sn)<2:
        tokens.append('$')
        main_words.append('$')
        continue
    else:
        main_words.append(sn[0])
        arr=sn[1].split('$')
        for i in range(len(arr)):
           
            arr[i]=arr[i].split('_')
            if 'CHEMICAL' in arr[i][0]:
                arr[i][0]='CHEMICAL'
                tokens.append('_'.join(arr[i]))
            elif 'GENE' in arr[i][0]:
                arr[i][0]='GENE'
                tokens.append('_'.join(arr[i]))
            elif 'SPECIES' in arr[i][0] :
                arr[i][0]='SPECIES'
                tokens.append('_'.join(arr[i]))
            elif 'DISEASE' in arr[i][0]:          
                arr[i][0]='DISEASE'
                tokens.append('_'.join(arr[i]))
            
            else:
                tokens.append('_'.join(arr[i]))
    tokens.append('$')

In [35]:
l = 'subEncode.'
encodeOutput = open(path + l + 'txt', 'r')
snID=0
tokensEncode=[]
for line in encodeOutput:
    sn = line.rstrip(' .\n').split('$\t$')
    snID+=1
    if len(sn)<2:
        if sn:
            tokensEncode.append(sn[0])
        else:
            tokensEncode.append('$')
        continue
    else:
        arr=sn[1].split('$')
        for i in range(len(arr)):
            tokensEncode.append(arr[i])
    tokensEncode.append('$')

In [36]:
print(len(tokens), len(tokensEncode), len(main_words))

4857365 4857365 1284856


In [37]:
if len(tokensEncode)!=len(tokens):
    for i in range(len(tokensEncode)):
        if (tokensEncode[i]=='$' and tokens[i]!='$') or (tokensEncode[i]!='$' and tokens[i]=='$'):
            print(tokens[i-10:i+10], tokens[i])
            print(tokensEncode[i-10:i+10], tokensEncode[i])
            print(i)
            break

In [38]:
f_out=path +'metapattern.txt'
frequentPatterns, pattern_location_dict, exact_match_dict=frequentPatternMining(tokens,tokensEncode,10,15)
write=True
if write:
    with open(f_out, 'w') as patternOutput:
        for p in frequentPatterns:
            line=p[0]+'\t'+str(p[1])
            patternOutput.write(line + '\n')



# of distinct tokens =  52488
working on length =  1
working on length =  2
working on length =  3
working on length =  4
working on length =  5
working on length =  6
working on length =  7
working on length =  8
working on length =  9
working on length =  10


In [39]:
f_out=path + 'pattern_match.txt'
k=0
pattern=[]
with open(f_out, 'w') as patternOutput:
    for j in range(len(tokensEncode)):
        
        if tokensEncode[j]=='$' and main_words[k]=='$':
            k+=1
            patternOutput.write('\n')
        elif tokensEncode[j]=='$' and main_words[k]!='$':
            
            patterns='\t'.join(pattern)
            patternOutput.write(main_words[k] + '\t' + patterns+'\n')
            pattern=[]
            k+=1
            #patternOutput.write('\n')
        elif tokensEncode[j]!='$' and tokens[j]=='$':
            k+=1
            patternOutput.write(tokensEncode[j] + '\n')
        else:
            if j in exact_match_dict:
                pattern_t=sorted(exact_match_dict[j], key=len, reverse=True)[0]
                if len(pattern)==0 or not pattern[-1] in pattern_t:
                    pattern.append(pattern_t)
                else:
                    pattern[-1]=pattern_t


In [40]:
j=0
for i in exact_match_dict:
    print(i,exact_match_dict[i])
    j+=1
    if j>10:
        break

2 ['of CHEMICAL']
35 ['of CHEMICAL', 'effect of CHEMICAL']
930 ['of CHEMICAL', 'level of CHEMICAL']
1161 ['of CHEMICAL']
1263 ['of CHEMICAL']
1332 ['of CHEMICAL', 'effect of CHEMICAL', 'the effect of CHEMICAL']
1357 ['of CHEMICAL', 'inhibition of CHEMICAL']
1400 ['of CHEMICAL', 'effect of CHEMICAL']
1435 ['of CHEMICAL', 'effect of CHEMICAL']
1887 ['of CHEMICAL', 'effect of CHEMICAL']
2224 ['of CHEMICAL', 'DISEASE of CHEMICAL']


In [41]:
f = 'metapattern.'
g = 'stopword.'
inputPattern = open(path + f + 'txt', 'r')
stopwordFile = open(path + g + 'txt', 'r')
ner_types=['DISEASE','GENE', 'CHEMICAL','SPECIES']
stopword=[]
for line in stopwordFile:
    sn = line.rstrip('\n')
    stopword.append(sn)
tokens=[]
main_words=[]

multiNER = []
singleNER = []
phrase = []
for line in inputPattern:
    sn = line.rstrip('\n').split('\t')
    main_pattern = sn[0]
    nerNo=0
    for ner in ner_types:
        if ner in main_pattern:
            nerNo+=1
            r=main_pattern.split(ner)
            nerNo+=len(r)-2
    if nerNo<2 and nerNo>0:
        #print(line)
        r=main_pattern.split(' ')
        wordNo=len(r)
        for word in r:
            if word in stopword:
                wordNo-=1
        if wordNo>1:
            singleNER.append([main_pattern,int(sn[1])])
    elif nerNo==0:
        r=main_pattern.split(' ')
        wordNo=len(r)
        for word in r:
            if word in stopword:
                wordNo-=1
        if wordNo>1:
            phrase.append([main_pattern,int(sn[1])])
    else:
        multiNER.append([main_pattern,int(sn[1])])

        
singleNER.sort(key=lambda x: x[1],reverse=True)
multiNER.sort(key=lambda x: x[1],reverse=True)
phrase.sort(key=lambda x: x[1],reverse=True)
singlePath=path + 'single.pattern.' + 'txt'
multiPath=path + 'multi.pattern.' + 'txt'
phrasePath=path + 'phrase.pattern.' + 'txt'
with open(singlePath, 'w') as single:
    for p in singleNER:
        single.write(p[0]+'\t'+str(p[1])+'\n')
with open(multiPath, 'w') as multi:
    for p in multiNER:
        multi.write(p[0]+'\t'+str(p[1])+'\n')     
with open(phrasePath, 'w') as ph:
    for p in phrase:
        ph.write(p[0]+'\t'+str(p[1])+'\n')
        

In [54]:
f = 'subSentence.'
g = 'pattern_match.'
subOutput = open(path + f + 'txt', 'r')
pattern_match = open(path + g + 'txt', 'r')
sentence=set()

sns=[]
snencode=[]
for line in subOutput:
    sns.append(line)
l = 'subEncode.'
encodeOutput = open(path + l + 'txt', 'r')
for line in encodeOutput:
    snencode.append(line.rstrip('\n'))

In [55]:
for line in sns:
    sn = line.rstrip('\n').split('$\t$')
    
    if len(sn)<2:
        continue
    else:
        arr=sn[1].split('$')
        tokens=[]
        for i in range(len(arr)):
           
            arr[i]=arr[i].split('_')
            if 'CHEMICAL' in arr[i][0]:
                arr[i][0]='CHEMICAL'
                tokens.append('_'.join(arr[i]))
            elif 'GENE' in arr[i][0]:
                arr[i][0]='GENE'
                tokens.append('_'.join(arr[i]))
            elif 'SPECIES' in arr[i][0] :
                arr[i][0]='SPECIES'
                tokens.append('_'.join(arr[i]))
            elif 'DISEASE' in arr[i][0]:          
                arr[i][0]='DISEASE'
                tokens.append('_'.join(arr[i]))
            
            else:
                tokens.append('_'.join(arr[i]))
    sentence.add(' '.join(tokens))

In [56]:
phrases=set()
pattern=set()
for line in pattern_match:
    sn = line.rstrip('\n').split('\t')
    
    if len(sn)<2:
        continue
    else:
        arr=sn[1]
        pattern.add(arr)
        if arr in sentence:
            phrases.add(arr)

In [58]:
h='complete_pattern.'
with open(path + h + 'txt', 'w') as ph:
    for p in phrases:
        ph.write(p+'\n')

In [59]:
singlePath=path + 'single.pattern.' + 'txt'
singlePattern = open(singlePath, 'r')
singleComplete=[]
mergepattern=[]
for line in singlePattern:
    sn = line.rstrip('\n').split('\t')
    arr=sn[0]
    if arr in phrases:
        singleComplete.append(line)
        mergepattern.append(arr)  

In [60]:
h='complete_single_pattern.'
with open(path + h + 'txt', 'w') as ph:
    for p in singleComplete:
        ph.write(p)

In [61]:
def replaceKeyWord(sentence, encode, patterns):
    ner_types=['DISEASE','GENE', 'CHEMICAL','SPECIES']
    new_sub=['']*len(sentence)
    change=False
    for i in range(len(sentence)-1,-1,-1):
        sn= sentence[i].split(' \t ')
        if sn[1] in patterns:
            change=True
            root=encode[i].split(' \t ')[0]
            for ner in ner_types: 
                if ner in sn[1]:
                    temp=sentence[i].split(' ')
                    temp[0]=ner
                    sentence[i]=' '.join(temp)
                    break
            
            for j in range(len(sentence)):
                subtree=encode[j].split(' ')
                where=-1
                if root in subtree:
                    where=subtree.index(root)
                    
                    
                if where>0:
                    temp=sentence[j].split(' ')
                    temp[where]=ner
                    sentence[j]=' '.join(temp)
    for i in range(len(sentence)):        
        new_sub[i]=sentence[i]+'\n'
    return new_sub,change

In [62]:
new_sub=[]
changed_sub=[]
h='new_sub1.'
pr=open(path + h + 'txt', 'w')
for i, line in enumerate(sns):      
    if i % 10000 == 0:
        print(i)
    sn = line.rstrip('\n').split('$\t$')
    if sn[0]=='':
        # replace and write
        sentence_encode=snencode[start:i]
        #break
        new_sub_t, changed =replaceKeyWord(sentence, snencode[start:i],mergepattern)
        #new_sub=new_sub+new_sub_t
        #new_sub.append(line)
        for subs in new_sub_t:
            pr.write(subs)
        pr.write(line)
        if changed:
            changed_sub.append(i)
    elif len(sn)<2:
        #new_sub.append(line)
        pr.write(line)
        sentence=[]
        start=i+1
    else:
        arr=sn[1].split('$')
        tokens=[sn[0]]
        tokens.append('\t')
        for i in range(len(arr)):
           
            arr[i]=arr[i].split('_')
            if 'CHEMICAL' in arr[i][0]:
                arr[i][0]='CHEMICAL'
                tokens.append('_'.join(arr[i]))
            elif 'GENE' in arr[i][0]:
                arr[i][0]='GENE'
                tokens.append('_'.join(arr[i]))
            elif 'SPECIES' in arr[i][0] :
                arr[i][0]='SPECIES'
                tokens.append('_'.join(arr[i]))
            elif 'DISEASE' in arr[i][0]:          
                arr[i][0]='DISEASE'
                tokens.append('_'.join(arr[i]))
            
            else:
                tokens.append('_'.join(arr[i]))
        sentence.append(' '.join(tokens))


pr.close()
        
k='changed_sub1.'
with open(path + k + 'txt', 'w') as ph:
    for p in changed_sub:
        ph.write(str(p)+'\n')

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000


In [63]:
multiPath=path + 'multi.pattern.' + 'txt'
complPath=path + 'complete_pattern.' + 'txt'
multiPattern = open(multiPath, 'r')
complPattern = open(complPath, 'r')
complMulti=[]
incomMulti=[]
phrases=[]
for line in complPattern:
    sn = line.rstrip('\n')
    phrases.append(sn) 

for line in multiPattern:
    sn = line.rstrip('\n').split('\t')
    arr=sn[0]
    if arr in phrases:
        complMulti.append(line)
    else:
        incomMulti.append(arr) 

In [64]:
complMulti

['GENE , GENE\t4652\n',
 'GENE and GENE\t4617\n',
 'CHEMICAL and CHEMICAL\t3716\n',
 'CHEMICAL , CHEMICAL\t2731\n',
 'GENE ( GENE\t2622\n',
 'CHEMICAL CHEMICAL\t2518\n',
 'CHEMICAL ( CHEMICAL\t2483\n',
 'GENE GENE\t2397\n',
 'GENE ( GENE )\t2198\n',
 'CHEMICAL ( CHEMICAL )\t1979\n',
 'SPECIES DISEASE\t1768\n',
 'CHEMICAL_induced DISEASE\t1752\n',
 'DISEASE SPECIES\t1263\n',
 'DISEASE DISEASE\t1123\n',
 'DISEASE in SPECIES\t1090\n',
 'GENE , and GENE\t974\n',
 'DISEASE and DISEASE\t928\n',
 'DISEASE ( DISEASE\t774\n',
 'CHEMICAL or CHEMICAL\t736\n',
 'SPECIES with DISEASE\t720\n',
 'SPECIES GENE\t700\n',
 'CHEMICAL DISEASE\t650\n',
 'CHEMICAL , and CHEMICAL\t646\n',
 'DISEASE ( DISEASE )\t591\n',
 'DISEASE , DISEASE\t497\n',
 'SPECIES DISEASE cell\t497\n',
 'GENE , GENE ,\t460\n',
 'CHEMICAL_treated SPECIES\t408\n',
 'CHEMICAL in SPECIES\t407\n',
 'GENE GENE ,\t374\n',
 'CHEMICAL , CHEMICAL ,\t351\n',
 'CHEMICAL CHEMICAL ,\t340\n',
 'the GENE ( GENE\t310\n',
 ', GENE , GENE\t300\n',
 'S

In [65]:
stopword

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [66]:
main_pattern

'young SPECIES'